In [1]:
#Import Dependencies
import pandas as pd
from sodapy import Socrata

In [2]:
#Import user credentials

from config import *

In [3]:
client = Socrata('data.lacity.org', App_Token, username=username,password=password)

In [14]:
results = client.get("k8cc-2d49", limit=500000)

In [19]:
results[:5]

[{'area': '01',
  'area_name': 'Central',
  'crm_cd': '997',
  'crm_cd_desc': 'TRAFFIC COLLISION',
  'cross_street': '2ND',
  'date_occ': '2010-04-20T00:00:00.000',
  'dr_no': '100109637',
  'location': 'ALAMEDA',
  'location_1': {'coordinates': [-118.2381, 34.0455], 'type': 'Point'},
  'premis_cd': '101',
  'premis_desc': 'STREET',
  'rpt_dist_no': '0138',
  'time_occ': '1235',
  'vict_age': '55',
  'vict_descent': 'B',
  'vict_sex': 'M'},
 {'area': '07',
  'area_name': 'Wilshire',
  'crm_cd': '997',
  'crm_cd_desc': 'TRAFFIC COLLISION',
  'cross_street': 'RIMPAU',
  'date_occ': '2010-12-21T00:00:00.000',
  'dr_no': '100719446',
  'location': 'PICO',
  'location_1': {'coordinates': [-118.337, 34.0488], 'type': 'Point'},
  'premis_cd': '101',
  'premis_desc': 'STREET',
  'rpt_dist_no': '0758',
  'time_occ': '2218',
  'vict_age': '37',
  'vict_descent': 'B',
  'vict_sex': 'M'},
 {'area': '01',
  'area_name': 'Central',
  'crm_cd': '997',
  'crm_cd_desc': 'TRAFFIC COLLISION',
  'cross_st

In [15]:
results_df = pd.DataFrame.from_records(results)

In [16]:
results_df[:5]

,area,area_name,crm_cd,crm_cd_desc,cross_street,date_occ,date_rptd,dr_no,location,location_1,mocodes,premis_cd,premis_desc,rpt_dist_no,time_occ,vict_age,vict_descent,vict_sex
0,01,Central,997,TRAFFIC COLLISION,2ND,2010-04-20T00:00:00.000,NaN,100109637,ALAMEDA,"{'type': 'Point', 'coordinates': [-118.2381, 3...",NaN,101,STREET,0138,1235,55,B,M
1,07,Wilshire,997,TRAFFIC COLLISION,RIMPAU,2010-12-21T00:00:00.000,NaN,100719446,PICO,"{'type': 'Point', 'coordinates': [-118.337, 34...",NaN,101,STREET,0758,2218,37,B,M
2,01,Central,997,TRAFFIC COLLISION,BROADWAY,2010-06-04T00:00:00.000,NaN,100111904,5TH,"{'type': 'Point', 'coordinates': [-118.2507, 3...",NaN,101,STREET,0153,2310,29,H,M
3,01,Central,997,TRAFFIC COLLISION,SAN PEDRO,2010-03-22T00:00:00.000,NaN,100108154,4TH,"{'type': 'Point', 'coordinates': [-118.2422, 3...",NaN,101,STREET,0148,1325,43,B,F
4,13,Newton,997,TRAFFIC COLLISION,ADAMS,2010-01-12T00:00:00.000,NaN,101304785,HILL,"{'type': 'Point', 'coordinates': [-118.2708, 3...",NaN,101,STREET,1321,1800,34,O,M


In [17]:
results_df.to_csv("collision.csv")

In [18]:
results_df.count()

area            418509
area_name       418509
crm_cd          418509
crm_cd_desc     418509
cross_street    400211
date_occ        418509
date_rptd        26232
dr_no           418509
location        418509
location_1      418509
mocodes         335246
premis_cd       418482
premis_desc     417294
rpt_dist_no     418509
time_occ        418509
vict_age        382396
vict_descent    412457
vict_sex        412902
dtype: int64